# The model represents a primal then its dual #

In [1]:
# Libraries to import
import gurobipy as gp
import itertools
from gurobipy import GRB
import numpy as np

In [2]:
met = ['A','B','C','D','E']

reac = ['v1','v2','v3','v4','v5','v6','v7']

LB =[0,-10,0,0,-10,-10,-10]
UB =[10,10,10,10,10,10,10]

SeM = {('A','v1'):-1,('A','v2'):0,('A','v3'):1,('A','v4'):0,('A','v5'):0,('A','v6'):0,('A','v7'):0,
       ('B','v1'):1,('B','v2'):-1,('B','v3'):0,('B','v4'):-1,('B','v5'):0,('B','v6'):0,('B','v7'):0,
       ('C','v1'):0,('C','v2'):1,('C','v3'):0,('C','v4'):0,('C','v5'):1,('C','v6'):0,('C','v7'):0,
       ('D','v1'):-1,('D','v2'):0,('D','v3'):0,('D','v4'):0,('D','v5'):0,('D','v6'):1,('D','v7'):0,
       ('E','v1'):1,('E','v2'):0,('E','v3'):0,('E','v4'):0,('E','v5'):0,('E','v6'):0,('E','v7'):1}

In [3]:
LBR = dict(zip(reac,LB))
UBR = dict(zip(reac,UB))

In [4]:
ob = [0,0,0,1,0,0,0]
m = gp.Model()

# Add Variables

v = m.addVars(reac,lb=-GRB.INFINITY,ub=GRB.INFINITY, obj=ob, name='v')

m.modelSense = GRB.MAXIMIZE

# Add Constr


m.addConstrs((gp.quicksum(SeM[i,j] * v[j] for j in reac) == 0 for i in met), name='FBA')
    
m.addConstrs((LBR[j] <= v[j] for j in reac), name='LB')

m.addConstrs((v[j] <= UBR[j] for j in reac), name='UB')


m.optimize()
#m.write('SolFBA_toy.lp')

if m.status == GRB.INFEASIBLE:
    m.computeIIS()
    m.write('FBA_toy3.ilp')  # 

if m.status == GRB.OPTIMAL:
    obj = m.getObjective()
    print('Max v4 Flux','->',obj.getValue())
    var = m.getAttr('X',m.getVars())
    duals = m.getAttr("Pi", m.getConstrs())
    const = m.getConstrs()
    print('------>')
    for i in range(len(reac)):
        print('flux:',reac[i],'->',var[i])
     

Using license file C:\Users\alexa\gurobi.lic
Academic license - for non-commercial use only - expires 2021-09-12
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 19 rows, 7 columns and 25 nonzeros
Model fingerprint: 0xc9205916
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 19 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.000000000e+01
Max v4 Flux -> 10.0
------>
flux: v1 -> 0.0
flux: v2 -> -10.0
flux: v3 -> 0.0
flux: v4 -> 10.0
flux: v5 -> 10.0
flux: v6 -> 0.0
flux: v7 -> 0.0


In [99]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
ob = [0,0,0,1,0,0,0]
b = [0,0,0,0,0]
S = np.array([[-1,0,1,0,0,0,0],
              [1,-1,0,-1,0,0,0],
              [0,1,0,0,1,0,0],
              [-1,0,0,0,0,1,0],
              [1,0,0,0,0,0,1]]) #transpose the matrix
LB =[0,-10,0,0,-10,-10,-10]
UB =[10,10,10,10,10,10,10]
yj =[1,1,1,1,1,1,1]

d = gp.Model()
meta = [i for i in range(len(b))]
react = [i for i in range(len(ob))]

# Add Variables
    # lambda
l = d.addVars(meta,lb=-GRB.INFINITY,ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name='lambda')
    # mu and rho
m = d.addVars(react,lb=-GRB.INFINITY,ub=GRB.INFINITY, vtype=GRB.CONTINUOUS,name='mu')
p = d.addVars(react,lb=-GRB.INFINITY,ub=GRB.INFINITY, vtype=GRB.CONTINUOUS,name='rho')
    # gamma and omega
g = d.addVars(react,lb=-GRB.INFINITY,ub=GRB.INFINITY, vtype=GRB.CONTINUOUS,name='gamma')
o = d.addVars(react,lb=-GRB.INFINITY,ub=GRB.INFINITY, vtype=GRB.CONTINUOUS,name='omega')
    # eta
e = d.addVar(lb=-GRB.INFINITY,ub=GRB.INFINITY,name='eta')


#d.setObjective((sum(b[i]*l[i] for i in meta)
                #- sum(LB[j]*g[j] for j in react)
                #+ sum(UB[j]*o[j] for j in react)
                #- 10*e),GRB.MINIMIZE)

d.setObjective((-sum(LB[j]*g[j] for j in react) + sum(UB[j]*o[j] for j in react) - 10*e),GRB.MINIMIZE)

# Add Constrs 

d.addConstrs((gp.quicksum(S.transpose()[i,j]*l[j] for j in meta)
              - m[i] 
              + p[i] 
               == ob[i] for i in react if i !=3)
             ,name='S_dual')

d.addConstr((gp.quicksum(S.transpose()[3,j]*l[j] for j in meta) 
            - m[3]
            + p[3]
            - e == ob[3]), name='Sdual_t') # 

# mu yj linearization

d.addConstrs((LB[j]*yj[j] <= g[j] for j in react),name='LB_gamma')
d.addConstrs((g[j] <= UB[j]*yj[j] for j in react),name='UB_gamma')


d.addConstrs((LB[j]*(1-yj[j]) <= m[j] for j in react),name='lb-m')

d.addConstrs((m[j] <= UB[j]*(1-yj[j])+g[j] for j in react), name='m-g')

# rho yj linearization

d.addConstrs((LB[j]*yj[j] <= o[j] for j in react),name='LB_omega')
d.addConstrs((o[j] <= UB[j]*yj[j] for j in react),name='UB_omega')


d.addConstrs((LB[j]*(1-yj[j]) <= p[j] for j in react),name='lb-p')

d.addConstrs((p[j] <= UB[j]*(1-yj[j])+o[j] for j in react), name='p-o')



#d.write('DUAL_FBA_toy6.lp')

d.optimize()


#if d.status == GRB.OPTIMAL:
    #obj = d.getObjective()
    #print('Max v4 Flux','->',obj.getValue())
    #vari = d.getAttr('X',d.getVars())
    #d_duals = d.getAttr("Pi", d.getConstrs())
    #print('------>')
    #for i in range(len(reac)):
        #print('flux:',reac[i],'->',var[i])

#if d.status in (GRB.INFEASIBLE,GRB.INF_OR_UNBD,GRB.UNBOUNDED):
    #d.computeIIS()
    #if d.IISMinimal:
        #print('IIS is minimal\n')
    #else:
        #print('IIS is not minimal\n')
        #print('\nThe folllowing constraint(s) cannot be satisfied:')
    #for c in d.getConstrs():
        #if c.IISConstr:
            #print('%s'%d.constrName)


#if m.status == GRB.INFEASIBLE:
    #m.computeIIS()
    #m.write('FBA_toy3.ilp')  # 


     

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 63 rows, 34 columns and 96 nonzeros
Model fingerprint: 0x022993f8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 63 rows and 34 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.000000000e+01


In [100]:
if d.status == GRB.OPTIMAL:
    dobj = d.getObjective()
    print('Max v4 Flux','->',dobj.getValue())
    vari = d.getAttr('X',d.getVars())
    d_duals = d.getAttr("Pi", d.getConstrs())
    print('------>')
    #for i in range(len(reac)):
        #print('flux:',reac[i],'->',var[i])

Max v4 Flux -> 10.0
------>


In [70]:
var

[0.0, -10.0, 0.0, 10.0, 10.0, 0.0, 0.0]

In [94]:
l_values = vari[:len(l)]
m_values = vari[len(l):len(l)+len(m)]
p_values = vari[len(l)+len(m):len(l)+len(m)+len(p)]
g_values = vari[len(l)+len(m)+len(p):len(l)+len(m)+len(p)+len(g)]
o_values = vari[len(l)+len(m)+len(p)+len(g):len(l)+len(m)+len(p)+len(g)+len(o)]

In [95]:
g_values

[10.0, 0.0, 10.0, 10.0, 0.0, 0.0, 0.0]

In [96]:
var

[0.0, -10.0, 0.0, 10.0, 10.0, 0.0, 0.0]

In [128]:
ob

[0, 0, 0, 1, 0, 0, 0]

In [135]:
def changecoeff(ob):
    for i in range(len(ob)):
        ob[i] = ob[i]*0
    return ob

In [141]:
w = [1,0,0,0]
w1 = [0,1,0,0,0]
w2 = [0,0,1,0,0,0,0]

#changecoeff(w)
changecoeff(w1)
#changecoeff(w2)

[0, 0, 0, 0, 0]